**Author : Anand Veerarahavan**



**STRING SIMILARITY**

**Using simple sequence matching**

In [1]:
from difflib import SequenceMatcher

def similar(a,b):
  return SequenceMatcher(None, a, b).ratio()

similar("route", "root")

0.6666666666666666

In [2]:
similar("groot", "root")

0.8888888888888888

In [3]:
similar("man","deep")

0.0

In [4]:
similar("good", "nice")

0.0

**Problems**

1. The above example only matches words based on their sequence and not the meaning of the words

2. Therefore it is difficult to find a set of words with the same meaning and different spellings.
Example : "Good" and "Nice" both mean the same

**Cosine Similarity**

This is why we use a technique called cosine similarity.

Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. 

![alt text](https://i2.wp.com/dataaspirant.com/wp-content/uploads/2015/04/cosine.png)

![alt text](https://datascience-enthusiast.com/figures/cosine_sim.png)

In [7]:
# dummy code
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u, v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u**2))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / np.dot(norm_u, norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

In [9]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

s1 = "This is a bar."
s2 = "A Pub is similar to a bar."
s3 = "Totally not related to the other two lines ."

vector1 = text_to_vector(s1)
print(vector1)
vector2 = text_to_vector(s2)
vector3 = text_to_vector(s3)
cosine1 = get_cosine(vector1, vector2)
cosine2 = get_cosine(vector2, vector3)
print ('Cosine_01:', cosine1)
print ('Cosine_02:', cosine2)

Counter({'This': 1, 'is': 1, 'a': 1, 'bar': 1})
Cosine_01: 0.5669467095138409
Cosine_02: 0.13363062095621217


In [10]:
from sklearn.metrics.pairwise import cosine_distances,cosine_similarity
import numpy as np

dog = np.asarray([[1.2, 0.9, 1.45]])
cat = np.asarray([[1.1, 0.1, 1.1 ]])
mouse = np.asarray([[0.1, -0.9, 0.01]])

print (cosine_similarity(dog,cat))
print (cosine_similarity(cat,mouse))
print (cosine_distances(mouse,dog))

[[0.92399995]]
[[0.02195964]]
[[1.35753829]]


### Levenshtein Distance

The Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.

In [11]:
cities = {"Pittsburgh":"Pennsylvania",
          "Tucson":"Arizona",
          "Cincinnati":"Ohio",
          "Albuquerque":"New Mexico",
          "Culpeper":"Virginia",
          "Asheville":"North Carolina",
          "Worcester":"Massachusetts",
          "Manhattan":"New York",
          "Phoenix":"Arizona",
          "Niagara Falls":"New York"} 

In [12]:
# Insertion of a single symbol. This means that we add a character to a string s. 
# Example: If we have the string s = "Manhatan", we can insert the character "t" to get the correct spelling

s = "Manhatan"
s = s[:5] + "t" + s[5:]
print(s)

Manhattan


In [13]:
# Deletion of a single symbol

s = "Mannhattan"
s = s[:2] + s[3:]
print(s)

Manhattan


In [14]:
# Substitution of a single symbol In the following example, we have to change the letter "o" into the letter "a" to get the correct spelling

s = "Manhatton"
s = s[:7] + "a" + s[8:]
s

'Manhattan'

In [15]:
"""
The minimum edit distance between the two strings "Mannhaton" and "Manhattan" 
corresponds to the value 3, as we need three basic editing operation to transform
the first one into the second one
"""
s = "Mannhaton"
s = s[:2] + s[3:]         # deletion
print(s)

s = s[:5] + "t" + s[5:]   # insertion
print(s)

s = s[:7] + "a" + s[8:]   # substitution
print(s)

Manhaton
Manhatton
Manhattan


The Levenshtein distance has the following properties:

* It is zero if and only if the strings are equal.
* It is at least the difference of the sizes of the two strings.
* It is at most the length of the longer string.
* Triangle inequality: The Levenshtein distance between two strings is no greater than the sum of their Levenshtein distances from a third string.


In [16]:
# recursive implementation of LD
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])
    return res
print(LD("Python", "Peithen"))

3


In [17]:
def iterative_levenshtein(s, t):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
    """
    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution
    for r in range(rows):
        print(dist[r])
    
 
    return dist[row][col]
print(iterative_levenshtein("flaw", "lawn"))

[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 1, 2, 3, 4]
[3, 2, 1, 2, 3]
[4, 3, 2, 1, 2]
2


In [18]:
print(iterative_levenshtein("Manhattan", "Manahaton"))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 1, 0, 1, 2, 3, 4, 5, 6, 7]
[3, 2, 1, 0, 1, 2, 3, 4, 5, 6]
[4, 3, 2, 1, 1, 1, 2, 3, 4, 5]
[5, 4, 3, 2, 1, 2, 1, 2, 3, 4]
[6, 5, 4, 3, 2, 2, 2, 1, 2, 3]
[7, 6, 5, 4, 3, 3, 3, 2, 2, 3]
[8, 7, 6, 5, 4, 4, 3, 3, 3, 3]
[9, 8, 7, 6, 5, 5, 4, 4, 4, 3]
3


In [19]:
! pip install python-Levenshtein

     |████████████████████████████████| 50 kB 3.6 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149862 sha256=7a80c9f983727182490e780942bdb344231ec37b14d543405aff3ae11684a908
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [20]:
import Levenshtein

"""
distance(string1, string2)
Compute absolute Levenshtein distance of two strings

"""
Levenshtein.distance("Exxon", "Exam")

3

In [21]:
Levenshtein.distance('Levenshtein', 'Lenvinsten')

4

# Median function

median(string_sequence[, weight_sequence])
Find an approximate generalized median string using greedy algorithm.

You can optionally pass a weight for each string as the second argument. 
The weights are interpreted as item multiplicities, 
although any non-negative real numbers are accepted. 
Use them to improve computation speed when strings often appear multiple times in the sequence.


In [22]:
Levenshtein.median(['SpSm', 'mpamm', 'Spam', 'Spa', 'Sua', 'hSam'])

'Spam'

In [23]:
fixme = ['Levnhtein', 'Leveshein', 'Leenshten',
...          'Leveshtei', 'Lenshtein', 'Lvenstein',
...          'Levenhtin', 'evenshtei']
Levenshtein.median(fixme)

'Levenshtein'

In [24]:
# fixme = ["beer", "bear","beare","beere","beeri","beire", "beer"]
fixme = ["beer", "bear","beire","beare","beaer","beiare", "beea"]
Levenshtein.median(fixme)

'bear'

The Levenshtein distance algorithm has been used in:

* Spell checking
* Speech recognition
* DNA analysis
* Plagiarism detection

amongst other applications.